# Tactus analysis

Analysis of Tactus AdB data for data paper (2020).

In [1]:
import os
import re
import sys
import xml.etree.ElementTree as ET
sys.path.insert(1, '/home/erikt/project/e-mental-health/data-processing')
import tactus2table

In [2]:
DATADIR = "/home/erikt/projects/e-mental-health/usb/tmp/20190917/"
INTAKE = "Intake"

## 1. How many clients have a therapy start date in their profile?

In [3]:
FILEPATTERN = r"^A.*z$"
QUERY1 = "./Treatment/StartDate"
STARTDATE = "STARTDATE"
NONE = "NONE"

def query1(root,query):
    try: text = root.findall(query)[0].text
    except: text = None
    if text is None: text = NONE
    elif text.strip() == "": text = NONE 
    else: text = STARTDATE
    return(text)

files = sorted(os.listdir(DATADIR))

data = {}
for inFileName in files:
    if re.search(FILEPATTERN,inFileName):
        root = tactus2table.readRootFromFile(DATADIR+inFileName)
        text1 = query1(root,QUERY1)
        if text1 in data: data[text1] += 1
        else: data[text1] = 1
data

{'STARTDATE': 923, 'NONE': 1060}

## 2. How many clients sent an email to the counselor?

In [4]:
FILEPATTERN = r"^A.*z$"
QUERY2 = "./Messages/Message/Sender"
CLIENT = "CLIENT"
CLIENTMAIL = "CLIENTMAIL"
NOCLIENTMAIL = "NOCLIENTMAIL"

def query2(root,query):
    clientMail = NOCLIENTMAIL
    for node in root.findall(query): 
        text = node.text.strip()
        if text == CLIENT:
            clientMail = CLIENTMAIL
            break
    return(clientMail)

files = sorted(os.listdir(DATADIR))

data = {CLIENTMAIL:0,NOCLIENTMAIL:0}
for inFileName in files:
    if re.search(FILEPATTERN,inFileName):
        root = tactus2table.readRootFromFile(DATADIR+inFileName)
        text2 = query2(root,QUERY2)
        data[text2] += 1
data

{'CLIENTMAIL': 1125, 'NOCLIENTMAIL': 858}

## 3. How many of the intake forms have the title Vragenlijst?

In [5]:
FILEPATTERN = r"^A.*z$"
QUERY3 = "./Intake/Questionnaire/Title"

def query3(root,query):
    try: text = root.findall(query)[0].text.strip()
    except: text = NONE
    return(text)

files = sorted(os.listdir(DATADIR))

data = {}
for inFileName in files:
    if re.search(FILEPATTERN,inFileName):
        root = tactus2table.readRootFromFile(DATADIR+inFileName)
        text3 = query3(root,QUERY3)
        if text3 in data: data[text3] += 1
        else: data[text3] = 1
data

{'Intake': 1937, 'Vragenlijst': 46}

## 4. Combination of 1, 2 and 3

In [6]:
data = {}
for inFileName in files:
    if re.search(FILEPATTERN,inFileName):
        root = tactus2table.readRootFromFile(DATADIR+inFileName)
        text1 = query1(root,QUERY1)
        text2 = query2(root,QUERY2)
        text3 = query3(root,QUERY3)
        text4 = " ".join([text1,text2,text3])
        if text4 in data: data[text4] += 1
        else: data[text4] = 1
data

{'STARTDATE CLIENTMAIL Intake': 770,
 'NONE NOCLIENTMAIL Intake': 716,
 'STARTDATE NOCLIENTMAIL Intake': 131,
 'NONE CLIENTMAIL Intake': 320,
 'NONE CLIENTMAIL Vragenlijst': 14,
 'NONE NOCLIENTMAIL Vragenlijst': 10,
 'STARTDATE CLIENTMAIL Vragenlijst': 21,
 'STARTDATE NOCLIENTMAIL Vragenlijst': 1}